## Setup & Installation
##### Prerequisites:
Before you begin, ensure you have:

##### Google Cloud SDK installed if you intend to use Google Cloud Services.
##### Appropriate permissions to install python packages and libraries.


In [83]:
!pip install streamlit
!pip install google-cloud
!pip install asyncio asyncpg cloud-sql-python-connector["asyncpg"]==1.2.3
!pip install numpy pandas
!pip install pgvector
!pip install langchain transformers
!pip install google-cloud-aiplatform
!pip install shapely
!pip install pg8000


**Note**: Installing packages globally may lead to conflicts. Use virtual environments (`venv` or `conda`) for isolation.

#### **2. Importing Necessary Libraries**

Ensure the following libraries are imported:


In [8]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
from google.cloud.sql.connector import Connector
import pg8000
import time
import os
import asyncio
import asyncpg
import numpy as np
import pandas as pd
import json
import vertexai
from vertexai.preview.language_models import TextGenerationModel
import sqlalchemy
from pgvector.asyncpg import register_vector

#### Project information

In [9]:
PROJECT_ID = os.getenv('PROJECT_ID')
LOCATION = os.getenv('LOCATION')
vertexai.init(project=PROJECT_ID, location=LOCATION)

#### Database Credentials:

Never hardcode database credentials. Instead, fetch them from environment variables:

In [11]:
# export PROJECT_ID="your_project_id"
# export LOCATION="your_location"


In [12]:
project_id = "YOUR Project ID"
region = "Your region"
instance_name = "CloudSQL instance name"

database_name = "DB NAME"
database_user = "YOUR USER NAME"      ###### change your own
database_password = "YOU PASSWORD"  ###### change your own

In [17]:
database_name = os.getenv('DATABASE_NAME')
database_user = os.getenv('DATABASE_USER')
database_password = os.getenv('DATABASE_PASSWORD')


In [18]:
os.environ["INSTANCE_CONNECTION_NAME"] = "project-ID:region:DB-instance-name"
os.environ["DB_USER"] = "YOUR USER NAME"
os.environ["DB_PASS"] = "YOUR PASSWORD"
os.environ["DB_NAME"] = "DB TABLE NAME"

In [19]:
print(os.getenv("INSTANCE_CONNECTION_NAME"))
print(os.getenv("DB_USER"))
print(os.getenv("DB_PASS"))
print(os.getenv("DB_NAME"))

project-ID:region:DB-instance-name
YOUR USER NAME
YOUR PASSWORD
DB TABLE NAME


In [20]:
# Load the JSON data
with open("ry_bug.json", "r") as f:
    data = json.load(f)
    

data_points = data["issues"]

# Extract the relevant fields and append to the list
rows = []
for point in data_points:
    custom_fields_str = ' '.join([f"{k}: {v}" for k, v in point["Custom Fields"].items()])
    content = point["Subject"] + " " + point["Description"] + " " + custom_fields_str
    rows.append([
        point["Ticket ID"], 
        point["Subject"], 
        point["Description"], 
        point["Resolution"], 
        point["Current Behavior"], 
        point["Solution"], 
        content
    ])

# Convert to DataFrame
columns = ["Ticket ID", "Subject", "Description", "Resolution", "Current Behavior", "Solution", "Content"]
df = pd.DataFrame(rows, columns=columns)

In [21]:
def prepare_instance(issue):
    """Prepare instance with task type, title, and content"""
    custom_fields_str = ' '.join([f"{k}: {v}" for k, v in issue["Custom Fields"].items()])
    
    content = issue["Subject"] + " " + issue["Description"] + " " + custom_fields_str + \
              " " + issue["Resolution"] + " "+ issue["Solution"]
        
    return {
        "task_type": "RETRIEVAL_DOCUMENT",
        "title": issue["Subject"],
        "content": content
    }

instances = [prepare_instance(issue) for issue in data["issues"]]

def chunkify(lst, chunk_size):
    """Break up the list lst into chunks of chunk_size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def get_embeddings_for_tickets(instances):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")
    all_embeddings = []

    for chunk in chunkify(instances, 5):
        embeddings_chunk = model.get_embeddings([instance['content'] for instance in chunk])
        all_embeddings.extend([list(embedding.values) for embedding in embeddings_chunk])

    return all_embeddings

ticket_embeddings = get_embeddings_for_tickets(instances)

def get_embedding_for_content(content):
    instance = {
        "task_type": "RETRIEVAL_DOCUMENT",
        "content": content
    }
    embedding = get_embeddings_for_tickets([instance])[0]
    return embedding

df['Embedding'] = df['Content'].apply(get_embedding_for_content)

In [22]:

# Implementing the search_similar_tickets function
def search_similar_tickets(query, ticket_embeddings, issues, top_n=5):
    query_instance = {
        "task_type": "RETRIEVAL_QUERY",
        "content": query
    }
    query_embedding = get_embeddings_for_tickets([query_instance])[0]
    
    # Calculate similarity between query and all tickets
    similarity_scores = [sum(a*b for a, b in zip(query_embedding, ticket_embedding)) for ticket_embedding in ticket_embeddings]
    
    # Get indices of top_n most similar tickets
    top_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    
    # Return the actual tickets corresponding to these indices
    return [issues[i] for i in top_indices]


In [23]:
query = "100M return loss"
similar_issues = search_similar_tickets(query, ticket_embeddings, data["issues"])

# Printing the top 5 similar issues
for issue in similar_issues:
    print(f"Ticket ID: {issue['Ticket ID']}")
    print(f"Subject: {issue['Subject']}")
    print(f"Description: {issue['Description']}")
    print(f"Customer Name: {issue['Customer Name']}")
    print(f"Solution: {issue['Solution']}")
    print("------------")

Ticket ID: TKT-1016
Subject: [HQA] [IEEE]100M return loss margin is less than 5%
Description: 100M return loss margin is less than 5%
Customer Name: Quality Assurance Team
Solution: Hardware adjustments and firmware tuning
------------
Ticket ID: TKT-1007
Subject: [HQA][IEEE]5G Return loss Fail.
Description: 1G Return loss Fail.

Fail item

Port 1 B
Port 5 B
Port 7 A 
Port 8 D
Port 10 B 
Port 13 B
Port 14 B
Port 17 B
Port 18 B
Port 19 B
Port 20 B
Port 21 B D
Port 22 B

Customer Name: Li Wei
Solution: Swapped out the faulty Ethernet controller chip on the identified units. Successful tests after replacement.
------------
Ticket ID: TKT-1003
Subject: [HQA] [IEEE]2nd Source FP1 transformer 1000 Base-T, Jitter MASTER Filtered failed
Description: 1000 Base-T, Jitter MASTER Filtered failed
Customer Name: Michael Johnson
Solution: NA (Pending further investigation)
------------
Ticket ID: TKT-1023
Subject: [HQA] [IEEE]2nd Source FP1 transformer 1000 Base-T, Jitter MASTER Filtered failed
Descr

In [25]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

COLUMNS = {
    "Ticket Number": 0,
    "Subject": 1,
    "Description": 2,
    "Resolution": 3,
    "Current Behavior": 4,
    "Solution": 5,
    "Content": 6,
    "Similarity": 7
}

def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    connector = Connector()
    return sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=lambda: connector.connect(
            os.getenv("INSTANCE_CONNECTION_NAME"),
            "pg8000",
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASS"),
            db=os.getenv("DB_NAME")
        )
    )

# def get_embedding_for_query(query: str):
#     model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")
#     instance = {"task_type": "RETRIEVAL_QUERY", "content": query}
#     return model.get_embeddings([instance])[0].values

def get_embedding_for_query(query: str):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")
    instance = {
        "instances": [{
            "task_type": "RETRIEVAL_QUERY",
            "content": query
        }]
    }
    embeddings = model.get_embeddings(instance)
    return embeddings[0].values



def search_similar_tickets(user_query: str, 
                           search_method: str = "cosine",
                           similarity_threshold: float = 0.001,
                           num_matches: int = 6) -> list:
    assert user_query, "⚠️ Please input a valid search query"

    qe = get_embedding_for_query(user_query)
    distance_function = {
        "cosine": "<=>",
        "inner_product": "<#>",
        "euclidean": "<->"
    }.get(search_method, "<=>")

    with connect_with_connector().connect() as db_conn:
        query = f"""
            WITH vector_matches AS (
                SELECT ticket_number, 1 - (embedding {distance_function} :embedding) AS similarity
                FROM ticket_data
                WHERE 1 - (embedding {distance_function} :embedding) > :similarity_threshold
                ORDER BY similarity DESC
                LIMIT :num_matches
            )
            SELECT 
                t.ticket_number, t.subject, t.description, t.resolution,
                t.current_behavior, t.solution, t.content, vm.similarity
            FROM ticket_data AS t
            JOIN vector_matches AS vm ON t.ticket_number = vm.ticket_number
        """

        parameters = {
            "embedding": "[" + ",".join(map(str, qe)) + "]",
            "similarity_threshold": similarity_threshold,
            "num_matches": num_matches
        }

        results = db_conn.execute(sqlalchemy.text(query), parameters).fetchall()
        matches = [
            {column_name: row[index] for column_name, index in COLUMNS.items()}
            for row in results
        ]

    return matches

# Sample Call
tickets = search_similar_tickets(user_query="100M isn't work well, reall bad")
print(tickets)


In [31]:
#Beautiful priting function, doesn't really matter.
def print_tickets(tickets: list):
    """
    Nicely prints the list of tickets.
    """
    for idx, ticket in enumerate(tickets, start=1):
        print("="*50)
        print(f"Ticket #{idx}")
        print("="*50)
        
        for key, value in ticket.items():
            # Beautify certain lengthy fields to display better.
            if key in ["Description", "Content", "Current Behavior", "Solution"]:
                value = "\n\t" + "\n\t".join(value.splitlines())

            # Format the similarity score to be more readable.
            if key == "Similarity":
                value = f"{value:.4f}"

            print(f"{key}: {value}")
        print("\n\n")

# Sample Call
print_tickets(tickets)

In [15]:
#Beautiful priting function, doesn't really matter.
def print_tickets(tickets: list):
    """
    Nicely prints the list of tickets.
    """
    for idx, ticket in enumerate(tickets, start=1):
        print("="*50)
        print(f"Ticket #{idx}")
        print("="*50)
        
        for key, value in ticket.items():
            # Beautify certain lengthy fields to display better.
            if key in ["Description", "Content", "Current Behavior", "Solution"]:
                value = "\n\t" + "\n\t".join(value.splitlines())

            # Format the similarity score to be more readable.
            if key == "Similarity":
                value = f"{value:.4f}"

            print(f"{key}: {value}")
        print("\n\n")

# Sample Call
print_tickets(tickets)

Ticket #1
Ticket Number: TKT-1003
Subject: [HQA] [IEEE]2nd Source FP1 transformer 1000 Base-T, Jitter MASTER Filtered failed
Description: 
	1000 Base-T, Jitter MASTER Filtered failed
Resolution: NA
Current Behavior: 
	During testing, the 1000 Base-T, Jitter MASTER Filtered failed.
Solution: 
	NA (Pending further investigation)
Content: 
	[HQA] [IEEE]2nd Source FP1 transformer 1000 Base-T, Jitter MASTER Filtered failed 1000 Base-T, Jitter MASTER Filtered failed Stage: PVT customer model: CM-9810 HW-Model: HW-MK2.8 HW - CPU Board: Intel i7-10700K HW - Main Board: MB-V2.2 SW - Boot Loader: BL-v3.2 DW - NOS/Diag Ver.: Diag-1.8.6
Similarity: 0.5825



Ticket #2
Ticket Number: TKT-1007
Subject: [HQA][IEEE]1G Return loss Fail.
Description: 
	1G Return loss Fail.
	
	Fail item
	
	Port 1 B
	Port 5 B
	Port 7 A 
	Port 8 D
	Port 10 B 
	Port 13 B
	Port 14 B
	Port 17 B
	Port 18 B
	Port 19 B
	Port 20 B
	Port 21 B D
	Port 22 B
Resolution: Replaced the defective Ethernet controller chip. Re-tested to co

In [16]:


# ---- Step 1: Use Vertex AI LLM to Generate Alternative Queries ----

def generate_alternative_queries(project_id: str, location: str, base_query: str, num_alternatives: int = 5):
    """
    Generate alternative queries using Vertex AI LLM.
    """
    
    # MAX_RETRIES = 5

    parameters = {
        "temperature": 0.5,
        "max_output_tokens": 512,
        # "top_p": 0.8,
        # "top_k": 40,
        # "stopSequence" : "Original Input:"
    }

    model = TextGenerationModel.from_pretrained("text-bison-32k")
    # prompt = f"Generate {num_alternatives} alternative search queries based on: {base_query}"
    prompt = f"""
Generate {num_alternatives} different versions of the given user question to enhance information retrieval.
Do not create anything that is unrelated to the original question, but only use the known inforamtion to form the well structurize question or ask from the differnt way as an QA engineer describing the issue.
try to give first two output using Traditional Chinese, the the rest using English.

you should only generate 5 lines of questions. First 2 in Traditional Chinese, and following 3 by English.


Original Input: CPU Overheating Issue
Output: 
CPU過熱問題
CPU渲染任務時過熱
CPU overheating during tasks
CPU software overheating
CPU temperate rises to 90°C

Original Input: LED Status Error on Port 44
Output: 
44端口LED問題
LED狀態錯誤
LED不正常
LED port 44 issue
44端口燈亮錯誤

Original Input: Hi-Pot Test Failure
Output:
高壓測試失敗
Hi-pot test issue
Hi-pot測試不通過
Hi-pot測試有問題
高壓測試不通過

(this time you will only generate 5 lines.)
Original Input: {base_query}
Output:
    """
    
    
    response = model.predict(prompt, **parameters)
    print("response: ", response)
    # Assuming the model returns alternatives separated by newlines
    queries = response.text.split('\n')[:num_alternatives]
    print("queries: ", queries)
    return queries

# ---- Step 2 & 3: Embed and Search Queries, Apply Voting Mechanism ----

def retrieve_and_rank_tickets(base_query: str, num_alternatives: int = 5, top_n: int = 5):
    alternative_queries = generate_alternative_queries(os.getenv("PROJECT_ID"), os.getenv("LOCATION"), base_query, num_alternatives)
    
    alternative_queries.append(base_query)

    # Aggregate results from all queries
    all_tickets = {}
    for query in alternative_queries:
        tickets = search_similar_tickets(query)
        
        # For each ticket, increase its score in all_tickets based on its similarity
        for ticket in tickets:
            ticket_number = ticket["Ticket Number"]
            similarity = ticket["Similarity"]
            print(query, "    ===> ", ticket_number, similarity)
            if ticket_number in all_tickets:
                all_tickets[ticket_number]["count"] += 1
                all_tickets[ticket_number]["cumulative_similarity"] += similarity
            else:
                all_tickets[ticket_number] = {
                    "ticket": ticket,
                    "count": 1,
                    "cumulative_similarity": similarity
                }
    
    # Rank by number of appearances and then by cumulative similarity
    ranked_tickets = sorted(all_tickets.values(), key=lambda x: (x["count"], x["cumulative_similarity"]), reverse=True)
    
    # Return the top N tickets based on the ranking
    return [ticket["ticket"] for ticket in ranked_tickets[:top_n]]

# ---- Test ----
user_query = "高壓測試失敗 以太網隔離失敗"
top_tickets = retrieve_and_rank_tickets(user_query)
for ticket in top_tickets:
    print(ticket)

response:   高壓測試失敗，以太網隔離失敗
     高壓測試失敗，以太網隔離測試失敗
     高壓測試不通過，以太網隔離測試失敗
     High-pot test fail, Ethernet isolation test fail
     Hi-pot test fail and Ethernet isolation fail

Original Input: 2000台伺服器已損壞
Output:
2000台伺服器已損壞
2000台伺服器損壞
2000台伺服器故障
2000 servers are broken
2000 servers are damaged

Original Input: 硬碟損壞
Output:
硬碟損壞
硬碟故障
硬碟壞軌
Hard drive corruption
Hard drive failure

Original Input: 伺服器過熱
Output:
伺服器過熱
伺服器溫度過高
伺服器散熱不良
Server overheating
Server temperature too high

queries:  [' 高壓測試失敗，以太網隔離失敗', '     高壓測試失敗，以太網隔離測試失敗', '     高壓測試不通過，以太網隔離測試失敗', '     High-pot test fail, Ethernet isolation test fail', '     Hi-pot test fail and Ethernet isolation fail']
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1003 0.5825122138382944
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1007 0.560131003162113
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1010 0.5698104075394201
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1011 0.5679488905318236
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1013 0.5764910303659511
 高壓測試失敗，以太網隔離失敗     ===>  TKT-1023 0.56857286296233
     

In [17]:

def analyze_bug_using_tickets(project_id: str, location: str, user_query: str, top_tickets: list):
    """
    Generate a detailed report based on the user query and top tickets using the Vertex AI LLM.
    """
    # Create the prompt structure
    tickets_str = ""
    for idx, ticket in enumerate(top_tickets, start=1):
        tickets_str += f"""
Ticket {ticket['Ticket Number']}:
- Subject: {ticket['Subject']}
- Description: {ticket['Description']}
- Resolution (if available): {ticket['Resolution']}
- Solution (if available): {ticket['Solution']}
- Current Behavior: {ticket['Current Behavior']}
-----
        """
    
    prompt = f"""
Based on the user's query:
"{user_query}"

And the related bug tickets:
{tickets_str}

Please perform the following tasks:

1. Summarize each ticket briefly.
2. Highlight which tickets are most relevant to the current user query.
3. Provide a detailed analysis of the potential issue based on these past occurrences. If similar issues have been encountered before, describe their nature and how they were resolved.
4. Recommend resolutions or further steps for the current issue based on this historical data. If a similar resolution isn't found, suggest preliminary steps for investigating and potentially resolving this issue.

Remember, precision and detail are crucial. Lives depend on this analysis.
    """

    # Set parameters for LLM
    parameters = {
        "temperature": 0.4,
        "max_output_tokens": 1024
    }

    # model = TextGenerationModel.from_pretrained("text-bison@001")
    model = TextGenerationModel.from_pretrained("text-bison-32k")
    response = model.predict(prompt, **parameters)
    
    return response.text

# Using the function
detailed_report = analyze_bug_using_tickets(os.getenv("PROJECT_ID"), os.getenv("LOCATION"), user_query, top_tickets)
print(detailed_report)


 Ticket Summary:

TKT-1003: 1000 Base-T, Jitter MASTER Filtered failed
TKT-1013: Interrupt signal INTn_CPLD_Master_A shows unexpected behavior during recovery.
TKT-1010: Device gets excessively hot when running CPU intensive tasks.
TKT-1023: Jitter MASTER filter test for 1000 Base-T failed.
TKT-1011: 3V3 rise time doesn't meet the RTL9301 specification.

Most Relevant Tickets:

TKT-1003 and TKT-1023 are most relevant to the current user query as they both describe failures related to 1000 Base-T testing.

Detailed Analysis:

The failures described in TKT-1003 and TKT-1023 are similar in nature, both involving the failure of the 1000 Base-T Jitter MASTER filter test. The root cause of these failures could be related to a faulty transformer or other components in the signal chain. It is also possible that the test setup or procedure was not followed correctly, leading to erroneous results.

In TKT-1010, the device was observed to overheat during CPU intensive tasks. This could be due to 

In [18]:
%%writefile streamlit_bug_ticket_analysis.py

import streamlit as st
from google.cloud import aiplatform, sql
from google.cloud.sql.connector import Connector

from typing import List
import os

import time
import asyncio
import asyncpg
import numpy as np
from pgvector.asyncpg import register_vector

import sqlalchemy


import vertexai
# from vertexai.language_models import TextGenerationModel
from vertexai.preview.language_models import TextGenerationModel
from vertexai.language_models import TextEmbeddingModel



# Constants
COLUMNS = {
    "Ticket Number": 0,
    "Subject": 1,
    "Description": 2,
    "Resolution": 3,
    "Current Behavior": 4,
    "Solution": 5,
    "Content": 6,
    "Similarity": 7
}  # Keeping it the same as in original code... mapping to postgresDB

project_id = "rust-ry"
region = "us-central1"
instance_name = "liteon-bug-tickets-demo"
database_name = "mybugticket"
database_user = "ry-admin"     
database_password = "rust123"  

# ---- Functions (as you provided, with minor modifications) ----
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    connector = Connector()
    return sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=lambda: connector.connect(
            os.getenv("INSTANCE_CONNECTION_NAME"),
            "pg8000",
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASS"),
            db=os.getenv("DB_NAME")
        )
    )

# def get_embedding_for_query(query: str):
#     model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")
#     instance = {"task_type": "RETRIEVAL_QUERY", "content": query}
#     return model.get_embeddings([instance])[0].values

def get_embedding_for_query(query: str):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")
    instance = {
        "instances": [{
            "task_type": "RETRIEVAL_QUERY",
            "content": query
        }]
    }
    embeddings = model.get_embeddings(instance)
    return embeddings[0].values

def search_similar_tickets(user_query: str, 
                           search_method: str = "cosine",
                           similarity_threshold: float = 0.001,
                           num_matches: int = 6) -> list:
    assert user_query, "⚠️ Please input a valid search query"

    qe = get_embedding_for_query(user_query)
    distance_function = {
        "cosine": "<=>",
        "inner_product": "<#>",
        "euclidean": "<->"
    }.get(search_method, "<=>")

    with connect_with_connector().connect() as db_conn:
        query = f"""
            WITH vector_matches AS (
                SELECT ticket_number, 1 - (embedding {distance_function} :embedding) AS similarity
                FROM ticket_data
                WHERE 1 - (embedding {distance_function} :embedding) > :similarity_threshold
                ORDER BY similarity DESC
                LIMIT :num_matches
            )
            SELECT 
                t.ticket_number, t.subject, t.description, t.resolution,
                t.current_behavior, t.solution, t.content, vm.similarity
            FROM ticket_data AS t
            JOIN vector_matches AS vm ON t.ticket_number = vm.ticket_number
        """

        parameters = {
            "embedding": "[" + ",".join(map(str, qe)) + "]",
            "similarity_threshold": similarity_threshold,
            "num_matches": num_matches
        }

        results = db_conn.execute(sqlalchemy.text(query), parameters).fetchall()
        matches = [
            {column_name: row[index] for column_name, index in COLUMNS.items()}
            for row in results
        ]

    return matches

# # Sample Call
# tickets = search_similar_tickets(user_query="100M isn't work well")
# print(tickets)

#beautiful priting function, in markdown format.
def print_tickets_markdown(tickets: list):
    """
    Prints the list of tickets in a markdown format.
    """
    for idx, ticket in enumerate(tickets, start=1):
        print("#" * 3 + f" Ticket #{idx}")
        
        for key, value in ticket.items():
            # Beautify certain lengthy fields to display better.
            if key in ["Description", "Content", "Current Behavior", "Solution"]:
                value = "\n\n" + "\n".join(f"- {line}" for line in value.splitlines())

            # Format the similarity score to be more readable.
            if key == "Similarity":
                value = f"{value:.4f}"

            print(f"**{key}**: {value}\n")
        print("---\n\n")

# Sample Call
# print_tickets_markdown(tickets)


# ---- Step 1: Use Vertex AI LLM to Generate Alternative Queries ----

def generate_alternative_queries(project_id: str, location: str, base_query: str, num_alternatives: int = 5, temperature: float = 0.25):
    """
    Generate alternative queries using Vertex AI LLM.
    """
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 512,
        # "top_p": 0.8,
        # "top_k": 40,
        # "stopSequence" : "Original Input:"
    }

    model = TextGenerationModel.from_pretrained("text-bison-32k")
    # prompt = f"Generate {num_alternatives} alternative search queries based on: {base_query}"
    prompt = f"""
Generate {num_alternatives} different versions of the given user question to enhance information retrieval.
Do not create anything that is unrelated to the original question, but only use the known inforamtion to form the well structurize question or ask from the differnt way as an QA engineer describing the issue.
try to give first two output using Traditional Chinese, the the rest using English.

you should only generate 5 lines of questions. and do not output other things.


Original Input: CPU Overheating Issue
Output: 
CPU過熱問題
CPU渲染任務時過熱
CPU overheating during tasks
CPU software overheating
CPU temperate rises to 90°C

Original Input: LED Status Error on Port 44
Output: 
44端口LED問題
LED狀態錯誤
LED不正常
LED port 44 issue
44端口燈亮錯誤

Original Input: Hi-Pot Test Failure
Output:
高壓測試失敗
Hi-pot test issue
Hi-pot測試不通過
Hi-pot測試有問題
高壓測試不通過


Original Input: {base_query}
Output:
    """
    
    
    response = model.predict(prompt, **parameters)
    print("response: ", response)
    # Assuming the model returns alternatives separated by newlines
    queries = response.text.split('\n')[:num_alternatives]
    print("queries: ", queries)
    return queries

# ---- Step 2 & 3: Embed and Search Queries, Apply Voting Mechanism ----

def retrieve_and_rank_tickets(base_query: str, num_alternatives: int = 5, top_n: int = 5, temperature: float = 0.25):
    alternative_queries = generate_alternative_queries(os.getenv("PROJECT_ID"), os.getenv("LOCATION"), base_query, num_alternatives, temperature)
    
    alternative_queries.append(base_query)

    # Aggregate results from all queries
    all_tickets = {}
    for query in alternative_queries:
        tickets = search_similar_tickets(query)
        
        # For each ticket, increase its score in all_tickets based on its similarity
        for ticket in tickets:
            ticket_number = ticket["Ticket Number"]
            similarity = ticket["Similarity"]
            print(query, "    ===> ", ticket_number, similarity)
            if ticket_number in all_tickets:
                all_tickets[ticket_number]["count"] += 1
                all_tickets[ticket_number]["cumulative_similarity"] += similarity
            else:
                all_tickets[ticket_number] = {
                    "ticket": ticket,
                    "count": 1,
                    "cumulative_similarity": similarity
                }
    
    # Rank by number of appearances and then by cumulative similarity
    ranked_tickets = sorted(all_tickets.values(), key=lambda x: (x["count"], x["cumulative_similarity"]), reverse=True)
    
    # Return the top N tickets based on the ranking
    return [ticket["ticket"] for ticket in ranked_tickets[:top_n]]


def analyze_bug_using_tickets(project_id: str, location: str, user_query: str, top_tickets: list):
    """
    Generate a detailed report based on the user query and top tickets using the Vertex AI LLM.
    """
    # Create the prompt structure
    tickets_str = ""
    for idx, ticket in enumerate(top_tickets, start=1):
        tickets_str += f"""
Ticket {ticket['Ticket Number']}:
- Subject: {ticket['Subject']}
- Description: {ticket['Description']}
- Resolution (if available): {ticket['Resolution']}
- Solution (if available): {ticket['Solution']}
- Current Behavior: {ticket['Current Behavior']}
-----
        """
    
    prompt = f"""
Based on the user's query:
"{user_query}"

And the related bug tickets:
{tickets_str}

Please perform the following tasks:

1. Summarize each ticket briefly.
2. Highlight which tickets are most relevant to the current user query.
3. Provide a detailed analysis of the potential issue based on these past occurrences. If similar issues have been encountered before, describe their nature and how they were resolved.
4. Recommend resolutions or further steps for the current issue based on this historical data. If a similar resolution isn't found, suggest preliminary steps for investigating and potentially resolving this issue.

Remember, precision and detail are crucial. Lives depend on this analysis.
    """

    # Set parameters for LLM
    parameters = {
        "temperature": 0.4,
        "max_output_tokens": 1024
    }

    # model = TextGenerationModel.from_pretrained("text-bison@001")
    model = TextGenerationModel.from_pretrained("text-bison-32k")
    response = model.predict(prompt, **parameters)
    
    return response.text

def analyze_bug_using_tickets_tc(project_id: str, location: str, user_query: str, top_tickets: list):
    """
    Generate a detailed report based on the user query and top tickets using the Vertex AI LLM.
    """
    # Create the prompt structure
    tickets_str = ""
    for idx, ticket in enumerate(top_tickets, start=1):
        tickets_str += f"""
Ticket {ticket['Ticket Number']}:
- Subject: {ticket['Subject']}
- Description: {ticket['Description']}
- Resolution (if available): {ticket['Resolution']}
- Solution (if available): {ticket['Solution']}
- Current Behavior: {ticket['Current Behavior']}
-----
        """
    
    prompt = f"""
Based on the user's query:
"{user_query}"

And the related bug tickets:
{tickets_str}

Please perform the following tasks:

1. Summarize each ticket briefly.
2. Highlight which tickets are most relevant to the current user query.
3. Provide a detailed analysis of the potential issue based on these past occurrences. If similar issues have been encountered before, describe their nature and how they were resolved.
4. Recommend resolutions or further steps for the current issue based on this historical data. If a similar resolution isn't found, suggest preliminary steps for investigating and potentially resolving this issue.

Remember, precision and detail are crucial. Lives depend on this analysis.
用台灣的語言回答，僅有專有名詞及專業術語使用英文，讓整體的語句越流暢越好
    """

    # Set parameters for LLM
    parameters = {
        "temperature": 0.4,
        "max_output_tokens": 1024
    }

    # model = TextGenerationModel.from_pretrained("text-bison@001")
    model = TextGenerationModel.from_pretrained("text-bison-32k")
    response = model.predict(prompt, **parameters)
    
    return response.text



# ... (above is all functions: connect_with_connector, get_embedding_for_query, search_similar_tickets... etc.)

# Streamlit UI

# Initialize Streamlit's session state for language choice
if 'selected_language' not in st.session_state:
    st.session_state.selected_language = "English"

st.title("Advanced Ticket Analysis System")
st.sidebar.header("Configuration")


# Move language selection to sidebar


# User Inputs
user_query = st.text_input("Enter your issue/query:", "Ripple Noise test fail")
search_method = st.sidebar.selectbox("Search Method", ["cosine", "inner_product", "euclidean"], index=0)
temperature = st.sidebar.slider("Temperature", 0.0, 0.9, 0.5)
num_matches = st.sidebar.slider("Number of Matches", 1, 20, 6)
similarity_threshold = st.sidebar.slider("Similarity Threshold", 0.0, 1.0, 0.001)
language_selection = st.sidebar.radio(
    "Choose Analysis Language", ["English", "Traditional Chinese"]
)

# If the Analyze button is pressed
if st.button("Analyze"):
    # Generate Alternative Queries
    alternative_queries = generate_alternative_queries(project_id, region, user_query, 5 , temperature)
    alternative_queries.append(user_query)

    # Search similar tickets for each query
    all_found_tickets = []
    for query in alternative_queries:
        tickets = search_similar_tickets(query, search_method, similarity_threshold, num_matches)
        all_found_tickets.extend(tickets)

    # Display matched ticket details in Streamlit
    st.subheader("Top Matched Ticket Details")
    for idx, ticket in enumerate(all_found_tickets[:10], start=1):
        st.markdown(f"### Ticket {idx}: {ticket['Ticket Number']}")
        st.write(f"**Subject**: {ticket['Subject']}")
        st.write(f"**Similarity Score**: {ticket['Similarity']:.4f}")

        # Use Streamlit's expander for each section
        with st.expander("Description"):
            st.write(ticket['Description'])

        if ticket.get('Resolution'):
            with st.expander("Resolution"):
                st.write(ticket['Resolution'])

        if ticket.get('Solution'):
            with st.expander("Solution"):
                st.write(ticket['Solution'])

        if ticket.get('Current Behavior'):
            with st.expander("Current Behavior"):
                st.write(ticket['Current Behavior'])

        # Can continue this for other fields if necessary
        st.write("---")

    

    # Analyze bug using top tickets
    st.subheader("Detailed Analysis and Recommendations")
    # Use the value of language_selection to decide the language
    if language_selection == "English":
        detailed_report = analyze_bug_using_tickets(project_id, region, user_query, all_found_tickets[:5])
        st.write(detailed_report)
    elif language_selection == "Traditional Chinese":
        detailed_report_tc = analyze_bug_using_tickets_tc(project_id, region, user_query, all_found_tickets[:5])
        st.write(detailed_report_tc)



st.warning("""Please review and validate AI-generated results. Human judgment is critical, especially in crucial situations.

           請查看並驗證人工智慧產生的結果。人類的判斷至關重要，尤其是在關鍵情況下 ver 0925 0823。""")

if __name__ == "__main__":
    st.write("Streamlit Application for Advanced Ticket Analysis")

Overwriting streamlit_bug_ticket_analysis.py


In [19]:
# !npm i localtunnel

In [32]:
!streamlit run streamlit_bug_ticket_analysis.py & npx localtunnel --port 8501

['project-ID:region:DB-instance-name']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/google/cloud/sql/connector/instance.py", line 388, in _refresh_task
    refresh_data = await refresh_task
  File "/opt/conda/lib/python3.10/site-packages/google/cloud/sql/connector/instance.py", line 312, in _perform_refresh
    metadata = await metadata_task
  File "/opt/conda/lib/python3.10/site-packages/google/cloud/sql/connector/refresh_utils.py", line 101, in _get_metadata
    resp = await client_session.get(url, headers=headers, raise_for_status=True)
  File "/opt/conda/lib/python3.10/site-packages/aiohttp/client.py", line 643, in _request
    resp.raise_for_status()
  File "/opt/conda/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1005, in raise_for_status
    raise ClientResponseError(
aiohttp.client_exceptions.ClientResponseError: 400, message='Bad Requ